<a href="https://colab.research.google.com/github/anurag1210/llm-zoomcamp-2025/blob/main/Agentic_Search_Tavili.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Loading the environment variables

import os
from openai import OpenAI
import re
import httpx
import os
from tavily import TavilyClient

os.environ["TAVILY_API_KEY"] = "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"

client = TavilyClient(
    # This is the default and can be omitted
    api_key=os.environ.get("TAVILY_API_KEY"),
)


In [13]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


'The Blackwell GPU features a custom TSMC 4NP process, 208 billion transistors, and a 10 TB/s interconnect. It supports GDDR7 and HBM3e memory, and offers up to 20 petaflops of FP4 performance.'

In [14]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?

"""

In [16]:
!pip install duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.2 MB/s eta 0:00:00


In [17]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results


for i in search(query):
    print(i)

https://www.easeweather.com/north-america/united-states/california/san-francisco/today
https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629
https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA
https://weather.com/weather/today/l/San+Francisco+CA?placeId=82b02ba7ed07c1ef0050ee6f71b8943f2cf24489c2a7645da321493514deb881
https://www.weather-us.com/en/california-usa/san-francisco
https://www.localconditions.com/us/san-francisco/california/weather/


In [18]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."

    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


In [19]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://www.easeweather.com/north-america/united-states/california/san-francisco/today


<body class="today-page desktop-device"><div class="blanket" id="blanket"></div><header class="header"> <div class="section-wide-wrap flex header-inner-wrap"> <a class="logo-wrap" href="https://www.easeweather.com"><img alt="easeweather.com" height="37" src="https://www.easeweather.com/images/easeweather.com.svg" title="easeweather.com" width="195"/></a> <div class="search-and-hamburger flex-top"> <div class="search-box-wrap"> <div class="search-wrap"><input class="search-box" id="search-box" placeholder="Search for a location"/><span class="search-x"></span></div> <div class="search-recent-locations"></div> </div><div class="search-button"></div> <div class="hamburger-icon"></div><div class="hamburger-content"> <div class="hamburger-title first-hamburger-title">Units</div> <div class="unit-changer hamburger_option"> <div data-type="c">°C</div> <label aria-label="Units" class="switch" for=

In [20]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)

print(f"Website: {url}\n\n")
print(weather_data)

Website: https://www.easeweather.com/north-america/united-states/california/san-francisco/today


Today weather in San Francisco San Francisco hourly forecast for today How accurate is our weather forecast for San Francisco? See how others rate our San Francisco forecast accuracy and help others by voting. View of San Francisco today Explore San Francisco live webcam feed to see how the weather is shaping up today in San Francisco. See what people are wearing, check out the sky, and get a sense of the atmosphere before you step outside. Stay informed and make the most of your day with our live weather updates from San Francisco today. Weather in San Francisco today The weather in San Francisco today is expected to be noticeably cooler than usual, with a forecast temperature of 15 ° F , compared to an average of 19 ° F for June 27th in recent years. Quick links What to wear today in San Francisco? clothing recommendations The weather today in San Francisco - FAQ Get accurate weather for

In [21]:
#Agentic Search

# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751049600, 'localtime': '2025-06-27 11:40'}, 'current': {'last_updated_epoch': 1751049000, 'last_updated': '2025-06-27 11:30', 'temp_c': 14.1, 'temp_f': 57.4, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 4.7, 'wind_kph': 7.6, 'wind_degree': 246, 'wind_dir': 'WSW', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 94, 'cloud': 0, 'feelslike_c': 13.9, 'feelslike_f': 57.0, 'windchill_c': 12.3, 'windchill_f': 54.1, 'heatindex_c': 12.8, 'heatindex_f': 55.0, 'dewpoint_c': 11.3, 'dewpoint_f': 52.3, 'vis_km': 6.4, 'vis_miles': 3.0, 'uv': 7.3, 'gust_mph': 6.6, 'gust_kph': 10.5}}


In [22]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.775,
        "lon": -122.4183,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1751049600,
        "localtime": "2025-06-27 11:40"
    },
    "current": {
        "last_updated_epoch": 1751049000,
        "last_updated": "2025-06-27 11:30",
        "temp_c": 14.1,
        "temp_f": 57.4,
        "is_day": 1,
        "condition": {
            "text": "Mist",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/143.png",
            "code": 1030
        },
        "wind_mph": 4.7,
        "wind_kph": 7.6,
        "wind_degree": 246,
        "wind_dir": "WSW",
        "pressure_mb": 1015.0,
        "pressure_in": 29.96,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 94,
        "cloud": 0,
        "feelslike_c": 13.9,
        "feelslike_f": 57.0,
        "windchill_c": 12.3,
        "windch